In [2]:
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel


In [6]:
model_path = "/content/drive/MyDrive/Zepto IDC Query Classification/l123-lora-fine-tuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)

base_model = AutoModelForSequenceClassification.from_pretrained("sentence-transformers/all-mpnet-base-v2", num_labels=2)
model = PeftModel.from_pretrained(base_model, model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device).eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 60.0 MB/s eta 0:00:00


L1


In [9]:
import faiss
import numpy as np

# Load your L1 embeddings (from .npy or memory)
l1_embeddings = np.load("/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/all_l1_embeddings.npy")

# Normalize (FAISS works better with normalized vectors + cosine similarity)
faiss.normalize_L2(l1_embeddings)

# Build FAISS index
index = faiss.IndexFlatIP(l1_embeddings.shape[1])  # IP = inner product (cosine if normalized)
index.add(l1_embeddings)

# Save the index
faiss.write_index(index, "/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/faiss_l1.index")

print(f"FAISS index built and saved. Total entries: {index.ntotal}")


FAISS index built and saved. Total entries: 31207


In [11]:
all_l1_df=pd.read_csv("/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/AllL1.csv")
all_l2_df=pd.read_csv("/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/AllL2.csv")
all_l3_df=pd.read_csv("/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/AllL3.csv")

In [12]:
def embed_query(text, tokenizer, model):
    model.eval()
    with torch.no_grad():
        encoded = tokenizer(
            [text],
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        )
        input_ids = encoded["input_ids"].to(model.device)
        attention_mask = encoded["attention_mask"].to(model.device)

        outputs = model.base_model.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True
        )

        cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token
        return cls_embedding.cpu().numpy()

In [13]:
index = faiss.read_index("/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/faiss_l1.index")


In [14]:
def search_l1(query, all_l1_df, tokenizer, model, index, k=10):
    input_text = f"L1 {query}"
    embedding = embed_query(input_text, tokenizer, model)
    faiss.normalize_L2(embedding)

    scores, indices = index.search(embedding, k)
    results = []
    for score, idx in zip(scores[0], indices[0]):
        category = all_l1_df.iloc[idx]["category"]
        product = all_l1_df.iloc[idx]["product_name"]
        results.append((round(score, 4), category, product))
    return results

In [15]:
query = "loreal hair colour 500"
top_results = search_l1(query, all_l1_df, tokenizer, model, index)

print(f"\n🔍 Top L1 categories for: '{query}'\n")
for score, category, product in top_results:
    print(f"[{score}] {category} (product: {product})")



🔍 Top L1 categories for: 'loreal hair colour 500'

[0.5570999979972839] Hair Care (product: tel 500ml)
[0.45669999718666077] Hair Care (product: 5l oi)
[0.44760000705718994] Makeup & Beauty (product: 200ml bottle)
[0.4269999861717224] Hair Care (product: olive oil 100 gm)
[0.42080000042915344] Makeup & Beauty (product: mixer mini)
[0.41679999232292175] Hair Care (product: olive oil 100gm)
[0.4106999933719635] Hair Care (product: badam roghan)
[0.40540000796318054] Hair Care (product: parachute almond)
[0.40380001068115234] Pharma & Wellness (product: complan 500)
[0.3982999920845032] Hair Care (product: badam rogen)


In [ ]:
import faiss
import numpy as np

# Load your L1 embeddings (from .npy or memory)
l2_embeddings = np.load("/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/all_l2_embeddings.npy")

# Normalize (FAISS works better with normalized vectors + cosine similarity)
faiss.normalize_L2(l2_embeddings)

# Build FAISS index
index = faiss.IndexFlatIP(l2_embeddings.shape[1])  # IP = inner product (cosine if normalized)
index.add(l2_embeddings)

# Save the index
faiss.write_index(index, "/content/drive/MyDrive/Zepto IDC Query Classification/final embeds/faiss_l2.index")

print(f"FAISS index built and saved. Total entries: {index.ntotal}")


FAISS index built and saved. Total entries: 34118
